In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta



# Read Data

In [2]:
# Tweets Data
df_tweets = pd.read_csv('data/combined_data.csv').iloc[:,1:]
df_tweets['Date'] = df_tweets.Time.apply(pd.to_datetime)
df_tweets = df_tweets.dropna(how='any')
df_tweets.drop(columns='Time', inplace =True)
df_tweets.head()

,Content,Date
0,"@haltman Little do they know, birds aren't rea...",2022-03-11 19:41:00
1,@waitbutwhy What if one atom at a time in your...,2022-03-11 00:31:00
2,@WholeMarsBlog @Erdayastronaut @CopSub This we...,2022-03-10 05:44:00
3,@FedorovMykhailo @OMarkarova You're welcome. W...,2022-03-09 21:49:00
4,"@PPathole @SpaceX Optimized, fully-reusable St...",2022-03-09 21:41:00


In [3]:
# Stocks Data
df_stocks = pd.read_csv('data/tsla_2.csv')
df_stocks.head()

,Date,Open,Adj Close
0,2017-01-03,42.972000,43.397999
1,2017-01-04,42.950001,45.397999
2,2017-01-05,45.284000,45.349998
3,2017-01-06,45.386002,45.801998
4,2017-01-09,45.793999,46.256001


In [4]:
def convert_date(x,h, m):
    x = str.split(x,sep='-')
    x = [int(item) for item in x]
    date = datetime(year=x[0],month=x[1],day=x[2],hour=h, minute=m, second=0)
    return date

df_open = df_stocks[['Date','Open']]
df_open['date'] = df_open.Date.apply(lambda row:convert_date(row, 9, 30))
df_open.drop(columns='Date', inplace=True)
df_open.rename(columns={'Open':'Price','date':'Date'}, inplace = True)

df_close = df_stocks[['Date','Adj Close']]
df_close['date'] = df_close.Date.apply(lambda row:convert_date(row, 16, 00))
df_close.drop(columns='Date', inplace=True)
df_close.rename(columns={'Adj Close':'Price','date':'Date'}, inplace = True)

frames = [df_open, df_close]
df_stocks = pd.concat(frames)
df_stocks = df_stocks.sort_values(by='Date')


In [5]:
# Date Range
df_date= pd.DataFrame()
df_date['Date'] = pd.bdate_range(start='2017-06-19', end='2022-03-12', freq='min')
df_date

,Date
0,2017-06-19 00:00:00
1,2017-06-19 00:01:00
2,2017-06-19 00:02:00
3,2017-06-19 00:03:00
4,2017-06-19 00:04:00
...,...
2486876,2022-03-11 23:56:00
2486877,2022-03-11 23:57:00
2486878,2022-03-11 23:58:00
2486879,2022-03-11 23:59:00


In [6]:
df = pd.merge(df_date,df_stocks,how='left', on='Date')
df = pd.merge(df, df_tweets, how='left', on='Date')

In [8]:
df.head(200)

,Date,Price,Content
0,2017-06-19 00:00:00,NaN,NaN
1,2017-06-19 00:01:00,NaN,NaN
2,2017-06-19 00:02:00,NaN,NaN
3,2017-06-19 00:03:00,NaN,NaN
4,2017-06-19 00:04:00,NaN,NaN
...,...,...,...
195,2017-06-19 03:15:00,NaN,NaN
196,2017-06-19 03:16:00,NaN,NaN
197,2017-06-19 03:17:00,NaN,NaN
198,2017-06-19 03:18:00,NaN,NaN
